In [ ]:
!pip install --quiet optuna

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate
from tensorflow.keras.optimizers import Adam, Adagrad

import optuna
from optuna import Trial

In [ ]:
data = pd.read_csv('apple_processed_data.csv')

# Assuming df is your DataFrame
lag = 5
addition = list()
data['ema'] = data['close'].ewm(span=lag).mean()
addition.append('ema')
for i in range(1, lag+1):
  temp = 'lag_' + str(i)
  data[temp] = data['close'].shift(i)
  addition.append(temp)
data = data.iloc[lag:]

addition.extend(['close', 'open', 'lowest', 'highest', 'total_vol', 'mean_vol', 'std_vol'])
numerical_features = data[addition]
target = data['is_up']

In [ ]:
scaler = MinMaxScaler()
scaled_numerical_features = scaler.fit_transform(numerical_features)


In [ ]:
input_dim = 10000

tokenizer = Tokenizer(num_words=input_dim, oov_token='<OOV>')
tokenizer.fit_on_texts(data['news'])
sequences = tokenizer.texts_to_sequences(data['news'])
padded_sequences = pad_sequences(sequences, maxlen=200)

In [ ]:
# Create train and test splits
X_train_num, X_test_num, X_train_news, X_test_news, y_train, y_test = train_test_split(
    scaled_numerical_features, padded_sequences, target, test_size=0.2, random_state=42)

## Setup model and optuna

In [ ]:
def build_model(units):
  numerical_input = Input(shape=(X_train_num.shape[1],), name='num_input')
  numerical_output = Dense(units, activation='relu')(numerical_input)

  news_input = Input(shape = (X_train_news.shape[1], ), name='news_input')
  embedding = Embedding(input_dim = input_dim, output_dim = units, input_length = X_train_news.shape[1])(news_input)
  lstm_out = LSTM(units)(embedding)
  concat = Concatenate()([numerical_output, lstm_out])
  output = Dense(1, activation = 'sigmoid')(concat)

  model = Model(
      inputs = [numerical_input, news_input],
      outputs = output
  )
  model.compile(optimizer=Adagrad(), loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
# Define Model
def objective(trial: Trial):
  units = trial.suggest_int('units', low=8, high=64)

  numerical_input = Input(shape=(X_train_num.shape[1],), name='num_input')
  numerical_output = Dense(units, activation='relu')(numerical_input)

  news_input = Input(shape = (X_train_news.shape[1], ), name='news_input')
  embedding = Embedding(input_dim = input_dim, output_dim = units, input_length = X_train_news.shape[1])(news_input)
  lstm_out = LSTM(units)(embedding)
  concat = Concatenate()([numerical_output, lstm_out])
  output = Dense(1, activation = 'sigmoid')(concat)

  model = Model(
      inputs = [numerical_input, news_input],
      outputs = output
  )
  model.compile(optimizer=Adagrad(), loss='binary_crossentropy', metrics=['accuracy'])
  history = model.fit([X_train_num, X_train_news], y_train, batch_size=16, epochs=6, validation_split=0.2)
  loss, accuracy = model.evaluate([X_test_num, X_test_news], y_test)
  return accuracy


In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

trial = study.best_trial

print("Accuracy: {}".format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2024-03-23 23:20:47,329] A new study created in memory with name: no-name-c7b6ffc8-99bb-4bb2-8571-84f29b39ab40


Epoch 1/6
89/89 [==============================] - 21s 178ms/step - loss: 0.6930 - accuracy: 0.4856 - val_loss: 0.6932 - val_accuracy: 0.4873
Epoch 2/6
89/89 [==============================] - 11s 125ms/step - loss: 0.6928 - accuracy: 0.5300 - val_loss: 0.6927 - val_accuracy: 0.5239
Epoch 3/6
89/89 [==============================] - 11s 121ms/step - loss: 0.6926 - accuracy: 0.5285 - val_loss: 0.6924 - val_accuracy: 0.5521
Epoch 4/6
89/89 [==============================] - 11s 129ms/step - loss: 0.6925 - accuracy: 0.5229 - val_loss: 0.6922 - val_accuracy: 0.5324
Epoch 5/6
89/89 [==============================] - 12s 130ms/step - loss: 0.6925 - accuracy: 0.5236 - val_loss: 0.6920 - val_accuracy: 0.5408
Epoch 6/6
14/14 [==============================] - 0s 29ms/step - loss: 0.6919 - accuracy: 0.5248


[I 2024-03-23 23:22:14,586] Trial 0 finished with value: 0.5247747898101807 and parameters: {'units': 56}. Best is trial 0 with value: 0.5247747898101807.


Epoch 1/6
89/89 [==============================] - 13s 120ms/step - loss: 0.6950 - accuracy: 0.5208 - val_loss: 0.6892 - val_accuracy: 0.5408
Epoch 2/6
89/89 [==============================] - 10s 108ms/step - loss: 0.6944 - accuracy: 0.5222 - val_loss: 0.6893 - val_accuracy: 0.5408
Epoch 3/6
89/89 [==============================] - 9s 97ms/step - loss: 0.6941 - accuracy: 0.5215 - val_loss: 0.6895 - val_accuracy: 0.5408
Epoch 4/6
89/89 [==============================] - 10s 108ms/step - loss: 0.6938 - accuracy: 0.5208 - val_loss: 0.6896 - val_accuracy: 0.5408
Epoch 5/6
89/89 [==============================] - 10s 107ms/step - loss: 0.6937 - accuracy: 0.5222 - val_loss: 0.6898 - val_accuracy: 0.5437
Epoch 6/6
14/14 [==============================] - 0s 25ms/step - loss: 0.6934 - accuracy: 0.5203


[I 2024-03-23 23:23:15,216] Trial 1 finished with value: 0.5202702879905701 and parameters: {'units': 30}. Best is trial 0 with value: 0.5247747898101807.


...

Epoch 1/6
89/89 [==============================] - 13s 121ms/step - loss: 0.6931 - accuracy: 0.5173 - val_loss: 0.6902 - val_accuracy: 0.5408
Epoch 2/6
89/89 [==============================] - 11s 119ms/step - loss: 0.6930 - accuracy: 0.5194 - val_loss: 0.6903 - val_accuracy: 0.5324
Epoch 3/6
89/89 [==============================] - 11s 119ms/step - loss: 0.6930 - accuracy: 0.5166 - val_loss: 0.6903 - val_accuracy: 0.5352
Epoch 4/6
89/89 [==============================] - 11s 121ms/step - loss: 0.6929 - accuracy: 0.5201 - val_loss: 0.6903 - val_accuracy: 0.5408
Epoch 5/6
89/89 [==============================] - 11s 120ms/step - loss: 0.6929 - accuracy: 0.5208 - val_loss: 0.6904 - val_accuracy: 0.5437
Epoch 6/6
14/14 [==============================] - 0s 26ms/step - loss: 0.6925 - accuracy: 0.5338


[I 2024-03-24 01:29:49,598] Trial 98 finished with value: 0.5337837934494019 and parameters: {'units': 40}. Best is trial 20 with value: 0.5495495200157166.


Epoch 1/6
89/89 [==============================] - 14s 129ms/step - loss: 0.6948 - accuracy: 0.4933 - val_loss: 0.6971 - val_accuracy: 0.4592
Epoch 2/6
89/89 [==============================] - 11s 123ms/step - loss: 0.6941 - accuracy: 0.4827 - val_loss: 0.6960 - val_accuracy: 0.4338
Epoch 3/6
89/89 [==============================] - 11s 124ms/step - loss: 0.6937 - accuracy: 0.4940 - val_loss: 0.6952 - val_accuracy: 0.4732
Epoch 4/6
89/89 [==============================] - 10s 114ms/step - loss: 0.6934 - accuracy: 0.4961 - val_loss: 0.6946 - val_accuracy: 0.5014
Epoch 5/6
89/89 [==============================] - 11s 120ms/step - loss: 0.6931 - accuracy: 0.5130 - val_loss: 0.6942 - val_accuracy: 0.4845
Epoch 6/6
14/14 [==============================] - 0s 28ms/step - loss: 0.6925 - accuracy: 0.5180


[I 2024-03-24 01:30:58,357] Trial 99 finished with value: 0.5180180072784424 and parameters: {'units': 41}. Best is trial 20 with value: 0.5495495200157166.


Accuracy: 0.5495495200157166
Best hyperparameters: {'units': 63}


In [ ]:
model = build_model(41)
history = model.fit([X_train_num, X_train_news], y_train, batch_size=16, epochs=6, validation_split=0.2)
loss, accuracy = model.evaluate([X_test_num, X_test_news], y_test)
print(loss, accuracy)

Epoch 1/6
89/89 [==============================] - 14s 127ms/step - loss: 0.7161 - accuracy: 0.4799 - val_loss: 0.7296 - val_accuracy: 0.4620
Epoch 2/6
89/89 [==============================] - 10s 112ms/step - loss: 0.7091 - accuracy: 0.4919 - val_loss: 0.7216 - val_accuracy: 0.4394
Epoch 3/6
89/89 [==============================] - 11s 122ms/step - loss: 0.7049 - accuracy: 0.4982 - val_loss: 0.7162 - val_accuracy: 0.4507
Epoch 4/6
89/89 [==============================] - 11s 123ms/step - loss: 0.7020 - accuracy: 0.5004 - val_loss: 0.7121 - val_accuracy: 0.4704
Epoch 5/6
89/89 [==============================] - 11s 123ms/step - loss: 0.7000 - accuracy: 0.4912 - val_loss: 0.7091 - val_accuracy: 0.4873
Epoch 6/6
14/14 [==============================] - 0s 26ms/step - loss: 0.6984 - accuracy: 0.5045
0.698421061038971 0.5045045018196106
